In [1]:
import pandas as pd
import folium
import requests
import folium
from folium.plugins import MarkerCluster

In [2]:
# -------------------------
# 1. 데이터 정리
# -------------------------
data = {
    'location': [
        '서울시 광진구 자양1동', '서울시 광진구 구의1동 640-6', '서울시 광진구 구의2동 75-1',
        '서울시 광진구 능동 군자동 49-1', '서울시 광진구 화양동 37-82', '서울시 광진구 광장동 188-2',
        '서울시 광진구 자양4동 43-44', '서울시 광진구 자양4동 72-1', '서울시 광진구 구의1동 252-95',
        '서울시 광진구 구의1동 246-10', '서울시 광진구 자양4동 57-153', '서울시 광진구 자양4동 8-8',
        '서울시 광진구 자양2동 680-63', '서울시 광진구 구의2동 63-7', '서울시 광진구 화양동 111-117',
        '서울시 광진구 화양동 18-12', '서울시 광진구 중곡4동 93-13', '서울시 광진구 중곡1동 647-16',
        '서울시 광진구 구의3동 587-6', '서울시 광진구 중곡3동 695', '서울시 광진구 강변역 2번출구',
        '서울시 광진구 자양동 621'
    ],
    'length_m': [
        5029, 4529, 1354,
        2232, 1846, 100,
        200, 70, 70,
        100, 250, 200,
        840, 240, 500,
        130, 100, 60,
        170, 270, 120,
        140
    ]
}

df = pd.DataFrame(data)

In [4]:
# -------------------------
# 2. Kakao API로 Geocoding
# -------------------------
headers = {"Authorization": f"KakaoAK {'key'}"}

def get_coordinates_kakao(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    params = {"query": address}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        result = response.json()
        if result['documents']:
            location = result['documents'][0]['address']
            return float(location['y']), float(location['x'])
        else:
            return None, None
    else:
        return None, None

# 주소를 좌표로 변환
locations = []
for loc in df['location']:
    lat, lon = get_coordinates_kakao(loc)
    locations.append((lat, lon))

df['lat'] = [loc[0] for loc in locations]
df['lon'] = [loc[1] for loc in locations]


In [5]:
# -------------------------
# 3. 길이 기반 반경 조정 함수
# -------------------------
def adjust_radius(length_m):
    if length_m > 3000:
        return 500  # 대규모: 반경 500m
    elif length_m > 1000:
        return 300  # 중규모: 반경 300m
    elif length_m > 500:
        return 150  # 소규모: 반경 150m
    else:
        return 70   # 아주 짧은 구간: 반경 70m

# 반경 열 추가
df['radius_m'] = df['length_m'].apply(adjust_radius)


In [6]:
# -------------------------
# 4. 지도 시각화
# -------------------------
map_center = [37.5385, 127.0825]
m = folium.Map(location=map_center, zoom_start=13)

# 위험지역 표시
for idx, row in df.iterrows():
    if pd.notnull(row['lat']) and pd.notnull(row['lon']):
        folium.Circle(
            location=(row['lat'], row['lon']),
            radius=row['radius_m'],
            color='red',
            fill=True,
            fill_opacity=0.4,
            popup=row['location']
        ).add_to(m)

# 지도 저장
m.save("gwangjin_sinkhole_map_adjusted.html")

print("조정된 반경으로 지도 생성 완료! (gwangjin_sinkhole_map_adjusted.html)")

조정된 반경으로 지도 생성 완료! (gwangjin_sinkhole_map_adjusted.html)


## 후보지 1

In [7]:

# 지도 시작 위치 (후보지 1)
center_lat, center_lon = 37.539176, 127.0942
m = folium.Map(location=[center_lat, center_lon], zoom_start=16)

# 후보지 1 (추천 좌표) - 빨간색
folium.Marker(
    [center_lat, center_lon],
    popup='후보지 1 (자양동 680-3 부근)',
    icon=folium.Icon(color='red', icon='star')
).add_to(m)



In [8]:
# 지하철역 (노란색 마커 추가)
subway_stations = [
    (37.535961, 127.094592, '강변역 (2호선)')  # 강변역 위치
]
for lat, lon, name in subway_stations:
    folium.Marker(
        [lat, lon],
        popup=name,
        icon=folium.Icon(color='black', icon='info-sign')
    ).add_to(m)

In [9]:
# 주요 지점 데이터
locations = {
    "주차장": [
         (37.5356993, 127.095765, '테크노마트 지하주차장'),
        (37.536473, 127.094857, '롯데마트 강변점 주차장')
    ],
    "사설 키즈카페": [
        (37.5388, 127.0947, '블럭고고')
    ],
    "주요 시설": [
        (37.536140950479, 127.09529347536, '테크노마트'),
        (37.535845, 127.0956464, '롯데마트'),
        (37.5356086, 127.0960487, '강변CGV'),
        (37.5369294, 127.0945979, '구의공원'),
        (37.5395254, 127.0938136, '서울내외의원')
    ]
}

In [10]:
# 나머지 (주차장, 키즈카페, 주요시설) 마커 추가
# 그대로 유지

# 주차장 (파란색 표지판)
for lat, lon, name in locations["주차장"]:
    folium.Marker(
        [lat, lon],
        popup=name,
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

In [11]:

# 사설 키즈카페 (노란색 + 반경 50m 원)
for lat, lon, name in locations["사설 키즈카페"]:
    folium.Marker(
        [lat, lon],
        popup=name,
        icon=folium.Icon(color='orange', icon='cloud')
    ).add_to(m)

In [12]:
# 주요 시설 (분홍색)
for lat, lon, name in locations["주요 시설"]:
    folium.Marker(
        [lat, lon],
        popup=name,
        icon=folium.Icon(color='pink')
    ).add_to(m)


In [13]:
# 결과 저장
# m.save('candidate1_map.html')

print("지도 생성 완료: candidate1_map.html")
m

지도 생성 완료: candidate1_map.html


## 후보지 2

In [14]:
# 지도 시작 위치 (후보지 2)
center_lat, center_lon = 37.545516, 127.103222
m = folium.Map(location=[center_lat, center_lon], zoom_start=16)

# 후보지 2 (추천 좌표) - 빨간색
folium.Marker(
    [center_lat, center_lon],
    popup='후보지 2 (구의동 251-45 일대)',
    icon=folium.Icon(color='red', icon='star')
).add_to(m)



In [15]:
# 지하철역 (검정색 마커)
subway_stations = [
    (37.545029, 127.103784, '광나루역 (5호선)')
]
for lat, lon, name in subway_stations:
    folium.Marker(
        [lat, lon],
        popup=name,
        icon=folium.Icon(color='black', icon='info-sign')
    ).add_to(m)



In [16]:
# 주요 지점 데이터
locations = {
    "주차장": [
        (37.5470002, 127.0959932, '구의야구공원 주차장'),
        (37.5489318, 127.0981692, '아차산배수지체육공원 주차장'),
        (37.54563, 127.103971, '신성빌딩 주차장')
    ],
    "사설 키즈카페": [
        # 없음 (반경 500m 이내)
    ],
    "주요 시설": [
        (37.5475573, 127.0977677, '광진숲나루'),
        (37.5461887, 127.1065122, '광진구민 체육센터'),
        (37.5518658, 127.1007676, '아차산 생태공원'),
        (37.5431842, 127.1170996, '광나루 한강공원'),
        (37.5444174,127.1033029, '광나루 지구대')

    ]
}



In [17]:
# 주차장 (파란색 표지판)
for lat, lon, name in locations["주차장"]:
    folium.Marker(
        [lat, lon],
        popup=name,
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)



In [18]:
# 사설 키즈카페 (주황색 마커만)
for lat, lon, name in locations["사설 키즈카페"]:
    folium.Marker(
        [lat, lon],
        popup=name,
        icon=folium.Icon(color='orange', icon='cloud')
    ).add_to(m)



In [19]:
# 주요 시설 (분홍색 마커)
for lat, lon, name in locations["주요 시설"]:
    folium.Marker(
        [lat, lon],
        popup=name,
        icon=folium.Icon(color='pink')
    ).add_to(m)


In [20]:

# 결과 저장
# m.save('candidate2_map.html')

print("지도 생성 완료: candidate2_map.html")
m

지도 생성 완료: candidate2_map.html


## 후보지 3

In [23]:
# 지도 시작 위치 (후보지 3)
center_lat, center_lon = 37.5423558, 127.1009475
m = folium.Map(location=[center_lat, center_lon], zoom_start=16)

# 후보지 2 (추천 좌표) - 빨간색
folium.Marker(
    [center_lat, center_lon],
    popup='후보지 3 (아차산로 540 일대)',
    icon=folium.Icon(color='red', icon='star')
).add_to(m)



In [24]:
# 지하철역 (검정색 마커)
subway_stations = [
    (37.545029, 127.103784, '광나루역 (5호선)')
]
for lat, lon, name in subway_stations:
    folium.Marker(
        [lat, lon],
        popup=name,
        icon=folium.Icon(color='black', icon='info-sign')
    ).add_to(m)



In [25]:
# 주요 지점 데이터
locations = {
    "아파트": [
      (37.5449476, 127.1030064, '아파트1'),
      (37.5441452, 127.1015274, '아파트2'),
      (37.5431792, 127.0995622, '아파트3'),
      (37.5425096, 127.0977483, '아파트4'),
      (37.5432801, 127.0974164, '아파트5'),
      (37.5425728, 127.0952471, '아파트6'),
      (37.5424654, 127.0962926, '아파트7'),
      (37.5406205, 127.0986045, '아파트8'),
      (37.5419395, 127.0996713, '아파트9'),
      (37.5419029, 127.1009815, '아파트10'),
      (37.5397145, 127.1000979, '아파트 11'),
      (37.5423975, 127.1020703, '아파트12'),
      (37.5414998, 127.1032682, '아파트13'),
      (37.5431583, 127.1041888, '아파트14'),
      (37.5431583, 127.1041888, '아파트15')



    ],
    "사설 키즈카페": [
        # 없음 (반경 500m 이내)
    ],
    "주요 시설": [
        (37.5414, 127.0973, '권소아과의원'),
        (37.5423975, 127.1020703, '글로벌 가정의학과의원'),
        (37.5444174,127.1033029, '광나루 지구대')

    ]
}



In [26]:
# 주차장 (파란색 표지판)
for lat, lon, name in locations["아파트"]:
    folium.Marker(
        [lat, lon],
        popup=name,
        icon=folium.Icon(color='cadetblue', icon='info-sign')
    ).add_to(m)



In [27]:
# 사설 키즈카페 (주황색 마커만)
for lat, lon, name in locations["사설 키즈카페"]:
    folium.Marker(
        [lat, lon],
        popup=name,
        icon=folium.Icon(color='orange', icon='cloud')
    ).add_to(m)



In [28]:
# 주요 시설 (분홍색 마커)
for lat, lon, name in locations["주요 시설"]:
    folium.Marker(
        [lat, lon],
        popup=name,
        icon=folium.Icon(color='pink')
    ).add_to(m)



In [29]:
# 결과 저장
# m.save('candidate2_map.html')

print("지도 생성 완료: candidate2_map.html")
m

지도 생성 완료: candidate2_map.html
